In [1]:
#pre installment
#!pip install python-dotenv requests

In [65]:
import os
import requests
import pandas as pd
import time
from dotenv import load_dotenv
load_dotenv()

#secret variables
CLIENT_ID = os.getenv("CLIENT_ID")
REFRESH_TOKEN = os.getenv("REFRESH_TOKEN")

In [164]:
def refresh_api_token():
    r = requests.post(
        "https://accounts.spotify.com/api/token",
        headers={"Authorization": f'Basic NDczOWRmNWFiOGY0NGI3ZDhhODkxNDgzYjVmNjBlM2I6ZDE3YmY0ZTAyMDQxNDQ3ZWE2NTNjNDcyMjE1MDkyMTU='},
        data={
            "grant_type": "refresh_token",
            "refresh_token": REFRESH_TOKEN,
        }
    )
    #expiration occur
    res = r.json()
    if "error" in res:
        print(res["error_description"])
        return None
    
    global access_token
    access_token = res["access_token"]

In [165]:
#global variables
global access_token
url_base = "https://api.spotify.com/v1/"
redirect_uri = "https://github.com/kfleung129/cs4480"
kpop_genres = "0JQ5DAqbMKFGvOw3O4nLAf"
jpop_genres = ""

In [166]:
#refresh access_token every re-run
refresh_api_token()

In [5]:
#read spotify chart dataset
df = pd.read_csv('charts.csv')

In [6]:
def get_albums(id):
    global access_token
    r = requests.get(
        f'https://api.spotify.com/v1/artists/{id}/albums',
        headers={"Authorization": f'Bearer {access_token}'}
    )
    res = r.json()
    
    #error handling
    if "error" in res:
        print(res["error"]["message"])
        return None
        
    return res

In [7]:
def get_tracks(id):
    global access_token
    r = requests.get(
        f'https://api.spotify.com/v1/albums/{id}/tracks',
        headers={"Authorization": f'Bearer {access_token}'}
    )
    res = r.json()
    
    #error handling
    if "error" in res:
        print(res["error"]["message"])
        return None
        
    return res

In [8]:
def get_tracks_play_count(id):
    global access_token
    r = requests.get(
        f'https://api.t4ils.dev/albumPlayCount?albumid={id}',
        headers={"Authorization": f'Bearer {access_token}'}
    )
    res = r.json()
    
    #error handling
    if "error" in res:
        print(res["error"]["message"])
        return None
        
    return res

In [125]:
def get_track(id):
    global access_token
    r = requests.get(
        f'https://api.spotify.com/v1/tracks/{id}/',
        headers={"Authorization": f'Bearer {access_token}'}
    )
    res = r.json()
    
    #error handling
    if "error" in res:
        print(res["error"]["message"])
        return None
    
    return res

In [170]:
def get_artist(id):
    global access_token
    r = requests.get(
        f'https://api.spotify.com/v1/artists/{id}/',
        headers={"Authorization": f'Bearer {access_token}'}
    )
    print(r.status_code)
    print(r.content)
    #error handling
    if "error" in res:
        print(res["error"]["message"])
        return None
    res = r.json()
    print(access_token)
    
    #401 access_token expired
    if r.status_code == 401:
        refresh_api_token()
        get_artist(id)
    
    
        
    return res

In [127]:
def search(name, _type):
    global access_token
    r = requests.get(
        f'https://api.spotify.com/v1/search?q={name}&type={_type}',
        headers={"Authorization": f'Bearer {access_token}'}
    )
    res = r.json()
    
    #error handling
    if "error" in res:
        print(res["error"]["message"])
        return None
        
    return res

In [128]:
def get_track_genres(id):
    track = get_track(id)
    track_artists = track['artists']
    res = ''
    for artist in track_artists:
        _id = artist['id']
        genres = get_artist(_id)['genres']
        #if no genre, continue to next loop
        if len(genres) == 0:
            continue
        res = res + ','.join(genres) + ','
    return res[0: len(res) - 1]

In [129]:
def append_genres_to_dataframe(df):
    genres_col = []
    for i in df.index:
        track_id = df['url'][i][df['url'][i].rindex('/')+1:]
        genres_result = get_track_genres(track_id)
        print(f'{i}: {genres_result}')
        # delay 30 second in every 100 spotify api request to prevent API rate limit exceeded
        if i != 0 and i % 100 == 0:
            time.sleep(30)
        genres_col.append(genres_result)
    df.insert(df.shape[1], "genres", genres_col)
    df.save_csv()

In [168]:
start_index = 0
for i in range(400, df.shape[0], 400):
    append_genres_to_dataframe(df[start_index: i])
    start_index += i
    print(f'Done {i}')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [98]:
modified_df = append_genres_to_dataframe(df[0: 400])

0: colombian pop,dance pop,latin pop,pop,latin pop,reggaeton,reggaeton colombiano,trap latino,urbano latino
1: latin pop,mexican pop,puerto rican pop,latin pop,reggaeton,reggaeton colombiano,trap latino,urbano latino
2: boy band,latin pop,reggaeton,urbano latino
3: reggaeton,reggaeton colombiano,trap latino,urbano latino,dance pop,pop,rap latina,trap queen,urbano latino
4: latin hip hop,reggaeton,trap latino,urbano latino
5: colombian pop,latin pop,reggaeton,reggaeton colombiano,urbano latino
6: cumbia pop
7: reggaeton,urbano latino,reggaeton,reggaeton colombiano,trap latino,urbano latino
8: colombian pop,latin pop,vallenato,colombian pop,dance pop,latin pop,pop
9: puerto rican pop,reggaeton,trap latino,urbano latino
10: electro latino,latin hip hop,latin pop,urbano latino,latin hip hop,reggaeton,trap latino,urbano latino
11: latin pop,mexican pop,latin pop,reggaeton,reggaeton colombiano,trap latino,urbano latino
12: latin pop,reggaeton,reggaeton colombiano,trap latino,urbano latino
13

124: edm,pop dance,uk dance,bboy,instrumental funk
125: chicha,cumbia chilena,cumbia villera
126: cumbia pop,cumbia villera
127: canadian pop,pop
128: cumbia pop
129: cumbia pop,cumbia villera
130: cuarteto,cumbia pop,cumbia villera
131: reggaeton,trap latino,urbano latino
132: dance pop,dancehall,pop,pop rap
133: reggaeton,reggaeton flow,trap boricua,trap latino,urbano latino,latin hip hop,reggaeton,trap latino,urbano latino,reggaeton,urbano latino,reggaeton,trap latino,urbano latino,reggaeton,urbano latino
134: modern rock,pop,pov: indie,rock
135: argentine rock,cuarteto,cumbia villera
136: dance pop,miami hip hop,pop,dembow,rap dominicano,cubaton
137: british soul,new jersey rap,pop rap,rap,southern hip hop,trap
138: cuarteto,cumbia villera
139: reggaeton,reggaeton colombiano,trap latino,urbano latino
140: permanent wave,pop,pop dance
141: reggaeton,reggaeton flow,trap latino,urbano latino,latin hip hop,reggaeton,trap latino,urbano latino
142: reggaeton,trap latino,urbano latino
143

270: downtempo,dream pop,indietronica
271: edm
272: canadian hip hop,canadian pop,hip hop,pop rap,rap,barbadian pop,pop,urban contemporary
273: canadian pop,pop
274: girl group,pop,talent show,hip hop,pop rap,r&b,southern hip hop,trap,trap soul
275: neo soul
276: australian dance
277: british soul,pop,pop soul,uk pop
278: pop,viral pop,pop,post-teen pop
279: australian hip hop,australian indie,australian pop,beatboxing
280: permanent wave,pop,pop dance
281: conscious hip hop,hip hop,north carolina hip hop,rap
282: pop rap,rap,southern hip hop,trap,viral trap
283: australian pop,candy pop,dance pop,girl group,post-teen pop
284: australian pop
285: pop rap,portland hip hop,rap,underground hip hop
286: pop,pop dance,tropical house,uk dance
287: danish pop,scandipop
288: pop,alt z,pop
289: australian dance,australian pop,pop
290: canadian pop,pop
291: modern rock,pop,pov: indie,rock
292: pop
293: electro house
294: hip hop,new orleans rap,pop rap,rap,trap,hip hop,pittsburgh rap,pop rap,rap

In [172]:
get_artist("0TnOYISbd1XYRBk9myaseg")

429
b'Too many requests'


UnboundLocalError: local variable 'res' referenced before assignment